In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
os.getcwd()

# S&P 500 成分股名稱

In [ ]:
#抓symbol時使用

#Abbreviate company name
sp_500 = pd.read_csv('WSJ S_P 500 成分股.csv')

#["Co.", "Corp.", "Cos."]
keyword = []
for i in sp_500["Company"]:
    company = i.replace(" Cl A", "").replace(" Cl B", "").replace(" Cl C", "")\
                .replace(" Series A","").replace(" Series C","")\
                .replace(" Inc.", "")\
                .replace(" PLC", "")\
                .replace(" Ltd.", "")\
                .replace(".com", "")\
                .replace(" N.A.", "")\
                .replace(" N.V.", "")
    keyword.append(company)
sp_500["Company"] = keyword

# Stock Price前處理

In [ ]:
datapath = []
#path = 'C:\\Users\\Julie\\Documents\\Python\\news\\daily'
path = 'C:\\Users\\Julie\\Documents\\Python\\news\\daily'
for dirname in os.listdir(path):#資料夾裡的子資料夾
    datapath.append(os.path.join(path, dirname))
print(len(datapath))
datapath[:5]

In [ ]:
for i in datapath:
    price = pd.read_csv(i)
    price.columns = price.iloc[0]#重設column名
    price.drop(0, inplace = True)#刪除column名原本的位置
    price.reset_index(inplace=True, drop = True)#重設index
    price.iloc[0,0] = datetime.strptime(price.iloc[0,0], "%m/%d/%Y").date()#格式錯誤，需改成日期
    price.to_csv(i, index = False)

In [ ]:
#轉換日期格式，並排序
for i in datapath:
    df = pd.read_csv(i)
    df.sort_values(by=['Date'], inplace = True, ignore_index = True)
    df.to_csv(i, index = False)

# Market Capital前處理

In [ ]:
datapath = []
#path = 'C:\\Users\\Julie\\Documents\\Python\\news\\S&P500_marketcap'
path = 'C:\\Users\\i7-870\\Documents\\Python\\news\\daily'
for dirname in os.listdir(path):#資料夾裡的子資料夾
    datapath.append(os.path.join(path, dirname))
print(len(datapath))
datapath[:5]

In [ ]:
for i in range(0, len(datapath)):
    print(i, datapath[i])
    data = pd.read_excel(datapath[i], usecols=["Date", "Market_Cap"])
    #修改日期格式
    data["Date"] = data["Date"].apply(lambda X: X.replace("March", "Mar").replace("April", "Apr").replace("June", "Jun").replace("July", "Jul").replace("Sept", "Sep").replace(".", ""))
    data["Date"] = data["Date"].apply(lambda X: datetime.strptime(X, "%b %d, %Y").date())
    data.sort_values(by=['Date'], inplace = True, ignore_index = True)#重新排序
    
    #修改市值格式
    value = []
    for j in data["Market_Cap"]:
        if "M" in j:
            j = float(j.replace("M", ""))
            value.append(int(j * 1000000))
        elif "B" in j:
            j = float(j.replace("B", ""))
            value.append(int(j * 1000000000))
        else:
            j = float(j.replace("T", ""))
            value.append(int(j * 1000000000000))
    data["Market_Cap"] = value
    
    path = "C:\\Users\\Julie\\Documents\\Python\\news\\S&P500_marketcap_new\\"
    data.to_csv(path + datapath[i].split("\\")[-1].split(".")[0] + ".csv", index = False)

# X-Negative Ratio檔名

In [ ]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for filename in dir_path:
        if "_e.csv" in filename:
            datapath.append(os.path.join(path, dirname, filename))
print(len(datapath))
datapath[:5]

# Concate X & Y to regression data

In [ ]:
reg_df = pd.DataFrame()
for i in range(len(datapath)):
    '''開啟News Ratio的檔案'''
    path1 = datapath[i]
    data = pd.read_csv(path1)
    
    news_columns = [0] + list(range(19,29))#news需保留的部分
    data = data.iloc[:, news_columns]
    data.rename(columns = {'date': 'Date'}, inplace = True)#方便merge
    
    
    '''抓出現在開的檔案的symbol'''
    Symbol_index = np.where( (path1.split("\\")[-1].split("_")[0]) == (sp_500["Company"]) )[0][0]
                            #np.where(條件):滿足條件輸出公司名稱的index
    symbol = sp_500["Symbol"][Symbol_index]
    print(i, symbol)
    
    
    '''透過symbol開啟stock price的檔案'''
    #path2 = 'C:\\Users\\Julie\\Documents\\Python\\news\\daily'
    path2 = 'C:\\Users\\i7-870\\Documents\\Python\\news\\daily'
    filepath = os.path.join(path2, symbol + " 30-12-2003 - 10-05-2021 d.csv")
    price = pd.read_csv(filepath)
    
    
    '''透過symbol開啟Market Capital的檔案'''
    #path2 = 'C:\\Users\\Julie\\Documents\\Python\\news\\S&P500_marketcap'
    path2 = 'C:\\Users\\i7-870\\Documents\\Python\\news\\S&P500_marketcap'
    filepath = os.path.join(path2, symbol + ".csv")
    market_cap = pd.read_csv(filepath)
    
    
    '''計算Return'''
    price["Return"] = np.log(price['Close'] / (price['Close'].shift(1)))
    price["Adjusted_Return"] = np.log10(price['Adj Close'] / (price['Adj Close'].shift(1)))
    price.dropna(axis = 0, inplace = True)#第一筆是Nan
    
    
    '''計算Volatility'''
    price["Volatility"] = ((np.log(price["High"]) - np.log(price["Close"])) * \
                       (np.log(price["High"]) - np.log(price["Open"]))) + \
                      ((np.log(price["Low"]) - np.log(price["Close"])) * \
                       (np.log(price["Low"]) - np.log(price["Open"])))
    
    
    '''計算Spread'''
    a = np.log(price["Low"]) - ( (np.log(price["Low"])+np.log(price["High"]))/2 )
    b = np.log(price["Low"]) - ( (np.log(price["Low"].shift(-1))+np.log(price["High"].shift(-1)))/2 )
    s = (4*(a*b))
    s = s.apply(lambda X: max(X, 0))
    price["Spread"] = s ** (1/2)
    
    
    '''clean Price :抓出2017-2020的資料'''
    price = price[["Date", "Return", "Adjusted_Return", "Volatility", "Spread", "Volume"]]#要保留的column
    try:
        index_start = np.where( price["Date"].apply(lambda X: "2017" in X) )[0][0]#2017的第一個
        index_end = np.where( price["Date"].apply(lambda X: "2021" in X) )[0][0]#2021的第一個
        price = price[index_start-5 : index_end]#lag 5，往前推5天
    except:#有些股價2017年後才有，所以index_start抓不到東西，直接從0開始
        index_end = np.where( price["Date"].apply(lambda X: "2021" in X) )[0][0]
        price = price[ : index_end]
    price.reset_index(drop = True, inplace = True)
    
    
    '''計算Volume'''
    #成交量的平均數為期間內，所以要先抓出需要的期間，才能計算
    price["Volume"] = (price["Volume"]-price["Volume"].mean())/price["Volume"].mean()
    
    
    '''計算Size'''
    market_cap["Size"] = np.log(market_cap["Market_Cap"] / 1000000)#轉成百萬單位
    market_cap.drop(["Market_Cap"], axis = 1, inplace = True)
    
    
    '''合併Control Variable'''
    #依照左邊的DataFrame的Date進行合併 (如果右邊的Date有，但左邊沒有，就不加入)
    control = pd.merge(left = price, right = market_cap, how='left', on='Date')
    control.dropna(inplace = True)
    control.reset_index(drop = True, inplace = True)
    
    
    '''合併News Variable & Control Variable'''
    df = pd.merge(left = control, right = data, how='left', on='Date')
       
    
    '''產生Variable的lag1 ~ lag5'''
    #產生新的lag1-lag5之column名
    new_column = df.columns.tolist()
    for j in range(1,6):
        for i in df.columns[1:]:
            new_column.append(i + "_lag" + str(j))
    
    #生成lag1-lag5
    df = pd.concat([df, df.iloc[:, 1:].shift(1), df.iloc[:, 1:].shift(2), df.iloc[:, 1:].shift(3),
                    df.iloc[:, 1:].shift(4), df.iloc[:, 1:].shift(5)], axis = 1)
    df = df.iloc[5:]#刪除前五筆None值(control variable)
    df.reset_index(drop = True, inplace = True)
    df.columns = new_column
    
    #刪除不需要的column
    del_column = df.loc[:, "Volatility":"text_sSubjectivity"].columns
    df.drop(del_column, axis = 1, inplace = True)
    
    #空值全補0
    df.fillna(0, inplace = True)

    df.to_csv(path1.replace("_e.csv", "_reg.csv"), index = False)
    
    
    '''concate regression data'''
    df.insert(loc = 0, column = "symbol", value = symbol)#增加一欄公司名稱
    reg_df = pd.concat([reg_df, df], axis = 0)
    reg_df.reset_index(drop = True, inplace = True)
    

reg_df.to_csv("reg_data.csv", index = False)

# Regression

In [ ]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import pandas as pd
from datetime import datetime

data = pd.read_csv("reg_data.csv")

In [ ]:
data.drop(['head_sNetural_ratio_lag1', 'head_sNetural_ratio_lag2', 'head_sNetural_ratio_lag3',
           'head_sNetural_ratio_lag4', 'head_sNetural_ratio_lag5',
           'text_sNetural_ratio_lag1', 'text_sNetural_ratio_lag2', 'text_sNetural_ratio_lag3',
           'text_sNetural_ratio_lag4', 'text_sNetural_ratio_lag5'], axis = 1,  inplace = True)

### 抓出要用的control variable

In [ ]:
flag = input("Y 為 Return -> 1 ； Y 為 Adj_Return -> 0: ") # 1 = Return；0 = Adj_Return
orignal = [i.replace("_lag1", "") for i in data.iloc[:, 4:10].columns]
#column 4-10是控制變數，先刪除變數的滯後名(_lag1)

#把沒有要用的Y刪掉
if int(flag) == 1: #1 = Return
    orignal.remove("Adj_Return")
else: #0 = Adj_Return
    orignal.remove("Return")

control = []
for j in orignal:
    for i in data.iloc[:, 4:].columns:#第四欄開始是控制變數
        if int(flag) != 1:
            if j in i:#ex: "Return" in "Return_lag2"
                control.append(i)
        else:
            if (j in i) & ("Adj_Return" not in i):#直接用Return下去找會找到Adj_Return，要先設條件
                control.append(i)
            
control

### Lag 1

In [ ]:
for i in range(10, 15, 4):
    
    Y = data[orignal[0]]
    
    x = list(data.columns)
    X1 =  data[[x[i]] + control]
    X2 =  data[[x[i + 1]] + control]
    X3 =  data[[x[i + 2]] + control]
    X4 =  data[[x[i + 3]] + control]
    
    results1 = sm.OLS(Y, sm.add_constant(X1)).fit()
    results2 = sm.OLS(Y, sm.add_constant(X2)).fit()
    results3 = sm.OLS(Y, sm.add_constant(X3)).fit()
    results4 = sm.OLS(Y, sm.add_constant(X4)).fit()

    df = summary_col([results1, results2, results3, results4], stars = True,
                     model_names = ["Positive", "Negative", "Polarity", "Subjectivity"],
                     regressor_order=["const"] + list(x[i: i+4]) + control)
    df = pd.DataFrame(df.tables[0])
    
    for j in range(len(df)):
        if (j % 2) == 1:
            df.iloc[j, 0] = df.iloc[j, 0] + "^"
            df.iloc[j, 1] = df.iloc[j, 1] + "^"
            df.iloc[j, 2] = df.iloc[j, 2] + "^"
            df.iloc[j, 3] = df.iloc[j, 3] + "^"
    
    if i == 10:
        path = "table/head_lag1.csv"
    else:
        path = "table/text_lag1.csv"
        
    df.to_csv(path)

### Lag 1-5

In [ ]:
#抓出要用的news variable
orignal_news = [i.replace("_lag1", "") for i in data.iloc[:, 10:20].columns]

news = []
for j in orignal_news:
    for i in data.iloc[:, 8:].columns:
        if j in i:
            news.append(i)

In [ ]:
for i in range(0, 21, 20):#news variable
    
    Y = data[orignal[0]]

    X1 =  data[news[i:i+5] + control]
    X2 =  data[news[i+5:i+10] + control]
    X3 =  data[news[i+10:i+15] + control]
    X4 =  data[news[i+15:i+20] + control]

    results1 = sm.OLS(Y, sm.add_constant(X1)).fit()
    results2 = sm.OLS(Y, sm.add_constant(X2)).fit()
    results3 = sm.OLS(Y, sm.add_constant(X3)).fit()
    results4 = sm.OLS(Y, sm.add_constant(X4)).fit()

    df = summary_col([results1, results2, results3, results4], stars = True,
                     model_names = ["Positive", "Negative", "Polarity", "Subjectivity"],                     
                     regressor_order=["const"] + list(news[i:i+21]) + control)
    df = pd.DataFrame(df.tables[0])
    
    
    for j in range(len(df)):
        if (j % 2) == 1:
            df.iloc[j, 0] = df.iloc[j, 0] + "^"
            df.iloc[j, 1] = df.iloc[j, 1] + "^"
            df.iloc[j, 2] = df.iloc[j, 2] + "^"
            df.iloc[j, 3] = df.iloc[j, 3] + "^"
    
    if i == 0:
        path = "table/head_lag1-5.csv"
    else:
        path = "table/text_lag1-5.csv"
        
    df.to_csv(path)

### Covid-19 compare 

In [ ]:
data["Date"] = data["Date"].apply(lambda X: datetime.strptime(X, "%Y-%m-%d").date())

In [ ]:
import datetime

before_covid = data[datetime.date(2019,12,1) >= data["Date"]]
after_covid = data[datetime.date(2019,12,1) <= data["Date"]]

before_covid.reset_index(drop = True, inplace = True)
after_covid.reset_index(drop = True, inplace = True)

In [ ]:
for i in before_covid.iloc[:, 10:18].columns:#news variable
    
    Y_before = before_covid[orignal[0]]
    X_before =  before_covid[[i] + control]
    
    Y_after = after_covid[orignal[0]]
    X_after =  after_covid[[i] + control]
    
    
    results1 = sm.OLS(Y_before, sm.add_constant(X_before)).fit()
    results2 = sm.OLS(Y_after, sm.add_constant(X_after)).fit()
    

    df = summary_col([results1, results2], model_names=["Before COVID-19", "During COVID-19"], stars = True)
    df = pd.DataFrame(df.tables[0])
    
    
    for j in range(len(df)):
        if (j % 2) == 1:
            df.iloc[j, 0] = df.iloc[j, 0] + "^"
            df.iloc[j, 1] = df.iloc[j, 1] + "^"
    
    
    path = "table/covid_" + i.replace("_lag1", "") + ".csv"
    df.to_csv(path)